In [ ]:
%%time
import os
from glob import glob
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline
import pandas as pd
import seaborn as sns
import re
import nltk
import json
import tensorflow as tf
from sklearn.utils import class_weight
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.preprocessing.text import Tokenizer
from sklearn.preprocessing import MultiLabelBinarizer
from sklearn.metrics import confusion_matrix
from sklearn.metrics import classification_report
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score,precision_score,recall_score,f1_score,roc_auc_score
from sklearn.metrics import average_precision_score,roc_auc_score, roc_curve, precision_recall_curve
from sklearn.preprocessing import LabelEncoder
np.random.seed(42)
import nltk
nltk.download('punkt')
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
from nltk.tokenize import ToktokTokenizer
import nltk, string, re, spacy,unicodedata, random
from bs4 import BeautifulSoup
import warnings
warnings.filterwarnings('ignore')

CPU times: user 52.1 ms, sys: 9.9 ms, total: 62 ms
Wall time: 123 ms


[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


In [ ]:
import os
from zipfile import ZipFile

# Unzip files
with ZipFile('/content/Train.zip', 'r') as zip_ref:
    zip_ref.extractall('/content/Train')

with ZipFile('/content/Dev.zip', 'r') as zip_ref:
    zip_ref.extractall('/content/Dev')

with ZipFile('/content/Test.zip', 'r') as zip_ref:
    zip_ref.extractall('/content/Test')

with ZipFile('/content/Test_labels.zip', 'r') as zip_ref:
    zip_ref.extractall('/content/Test_labels')
# List files in the Train directory
os.listdir('/content/Test_labels')

['431.jpg',
 '205.jpg',
 '467.jpg',
 '861.jpg',
 '136.jpg',
 '246.jpg',
 '852.jpg',
 '695.jpg',
 '939.jpg',
 '252.jpg',
 '774.jpg',
 '710.jpg',
 '35.jpg',
 '622.jpg',
 '254.jpg',
 '952.jpg',
 '984.jpg',
 '566.jpg',
 '510.jpg',
 '832.jpg',
 '311.jpg',
 '742.jpg',
 '806.jpg',
 '234.jpg',
 '874.jpg',
 '258.jpg',
 '667.jpg',
 '511.jpg',
 '190.jpg',
 '86.jpg',
 '67.jpg',
 '417.jpg',
 '353.jpg',
 '269.jpg',
 '11.jpg',
 '479.jpg',
 '31.jpg',
 '123.jpg',
 '646.jpg',
 '427.jpg',
 '176.jpg',
 '171.jpg',
 '857.jpg',
 '432.jpg',
 '209.jpg',
 '409.jpg',
 '851.jpg',
 '425.jpg',
 '73.jpg',
 '398.jpg',
 '794.jpg',
 '681.jpg',
 '673.jpg',
 '873.jpg',
 '27.jpg',
 '474.jpg',
 '210.jpg',
 '858.jpg',
 '297.jpg',
 '789.jpg',
 '687.jpg',
 '871.jpg',
 '830.jpg',
 '325.jpg',
 '157.jpg',
 '867.jpg',
 '954.jpg',
 '973.jpg',
 '326.jpg',
 '206.jpg',
 '445.jpg',
 '63.jpg',
 '760.jpg',
 '950.jpg',
 '293.jpg',
 '763.jpg',
 '360.jpg',
 '433.jpg',
 '571.jpg',
 '563.jpg',
 '501.jpg',
 '387.jpg',
 '639.jpg',
 '22.jpg',
 

In [ ]:
import pandas as pd
import numpy as np
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from sklearn.preprocessing import LabelEncoder
from sklearn.utils.class_weight import compute_class_weight
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Embedding, Bidirectional, LSTM, Dense, Dropout
from tensorflow.keras.regularizers import l2
from sklearn.metrics import classification_report

# Load the datasets (adjust the file path as needed)
train_csv = pd.read_csv('/content/Train/train.csv', encoding='utf-8')
test_csv = pd.read_csv('/content/Test/test.csv', encoding='utf-8')
dev_csv = pd.read_csv('/content/Dev/dev.csv', encoding='utf-8')
test_labels_csv=pd.read_csv('/content/Test_labels/test_with_labels.csv')
# Preprocess data
text_column = 'transcriptions'
label_column = 'labels'

# Fill missing values in 'transcriptions' column
train_csv[text_column] = train_csv[text_column].fillna('')
test_csv[text_column] = test_csv[text_column].fillna('')

# Tokenization
max_vocab_size = 10000
max_sequence_length = 100

tokenizer = Tokenizer(num_words=max_vocab_size, oov_token="<OOV>")
tokenizer.fit_on_texts(train_csv[text_column])

X_train = tokenizer.texts_to_sequences(train_csv[text_column])
X_test = tokenizer.texts_to_sequences(test_csv[text_column])
X_dev = tokenizer.texts_to_sequences(dev_csv[text_column])
X_train = pad_sequences(X_train, maxlen=max_sequence_length, padding='post', truncating='post')
X_test = pad_sequences(X_test, maxlen=max_sequence_length, padding='post', truncating='post')
X_dev = pad_sequences(X_dev, maxlen=max_sequence_length, padding='post', truncating='post')


# Label Encoding
label_encoder = LabelEncoder()
y_train = label_encoder.fit_transform(train_csv[label_column])
y_dev = label_encoder.transform(dev_csv[label_column])

# Compute class weights for binary classification
class_weights = compute_class_weight(
    class_weight='balanced',
    classes=np.unique(y_train),  # Use original binary labels (0 and 1)
    y=y_train
)
class_weights_dict = {i: class_weights[i] for i in range(len(class_weights))}

# Define the BiLSTM model with regularization
model = Sequential([
    Embedding(input_dim=max_vocab_size, output_dim=128, input_length=max_sequence_length),  # Embedding layer
    Bidirectional(LSTM(64, return_sequences=False)),  # BiLSTM layer
    Dropout(0.8),  # Dropout to avoid overfitting
    Dense(128, activation='relu', kernel_regularizer=l2(0.01)),  # Fully connected layer with L2 regularization
    Dropout(0.5),
    Dense(1, activation='sigmoid')  # Sigmoid for binary classification
])

# Compile the model
model.compile(
    optimizer='adam',
    loss='binary_crossentropy',  # Binary classification loss
    metrics=['accuracy']
)

# Model summary
model.build(input_shape=(None, max_sequence_length))  # Explicitly build the model with input shape
model.summary()

# Split the dev set (you already have dev_csv)
X_dev = tokenizer.texts_to_sequences(dev_csv[text_column])
X_dev = pad_sequences(X_dev, maxlen=max_sequence_length, padding='post', truncating='post')
y_dev = label_encoder.transform(dev_csv[label_column])

# Train the model
history = model.fit(
    X_train, y_train,
    validation_data=(X_dev, y_dev),
    epochs=10,
    batch_size=16,  # Lower batch size for better optimization in smaller datasets
    class_weight=class_weights_dict,  # Add class weights if class imbalance exists
    verbose=1
)

# Evaluate the model on the dev set using BiLSTM
val_loss, val_accuracy = model.evaluate(X_dev, y_dev)
print(f"Validation Loss: {val_loss}, Validation Accuracy: {val_accuracy}")

# Make predictions on the dev set
y_dev_pred = model.predict(X_dev)
y_dev_labels = (y_dev_pred > 0.5).astype(int)

# Classification report
print(classification_report(y_dev, y_dev_labels))




Model: "sequential_4"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━┓
┃ Layer (type)                         ┃ Output Shape                ┃         Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━┩
│ embedding_4 (Embedding)              │ (None, 100, 128)            │       1,280,000 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ bidirectional_4 (Bidirectional)      │ (None, 128)                 │          98,816 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dropout_8 (Dropout)                  │ (None, 128)                 │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_8 (Dense)                      │ (None, 128)                 │          16,512 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dropout_9 (Dropout)                  │ (None, 128)                 │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_9 (Dense)                      │ (None, 1)                   │             129 │
└──────────────────────────────────────┴─────────────────────────────┴─────────────────┘

 Total params: 1,395,457 (5.32 MB)

 Trainable params: 1,395,457 (5.32 MB)

 Non-trainable params: 0 (0.00 B)

Epoch 1/10
40/40 ━━━━━━━━━━━━━━━━━━━━ 11s 167ms/step - accuracy: 0.4245 - loss: 1.7759 - val_accuracy: 0.6187 - val_loss: 1.2774
Epoch 2/10
40/40 ━━━━━━━━━━━━━━━━━━━━ 8s 101ms/step - accuracy: 0.5480 - loss: 1.1788 - val_accuracy: 0.4938 - val_loss: 0.9585
Epoch 3/10
40/40 ━━━━━━━━━━━━━━━━━━━━ 7s 151ms/step - accuracy: 0.7580 - loss: 0.8332 - val_accuracy: 0.7250 - val_loss: 0.6941
Epoch 4/10
40/40 ━━━━━━━━━━━━━━━━━━━━ 8s 102ms/step - accuracy: 0.9591 - loss: 0.5003 - val_accuracy: 0.7312 - val_loss: 0.6980
Epoch 5/10
40/40 ━━━━━━━━━━━━━━━━━━━━ 7s 160ms/step - accuracy: 0.9918 - loss: 0.1568 - val_accuracy: 0.7812 - val_loss: 0.7866
Epoch 6/10
40/40 ━━━━━━━━━━━━━━━━━━━━ 4s 102ms/step - accuracy: 0.9999 - loss: 0.0947 - val_accuracy: 0.7437 - val_loss: 0.8615
Epoch 7/10
40/40 ━━━━━━━━━━━━━━━━━━━━ 5s 104ms/step - accuracy: 0.9998 - loss: 0.0679 - val_accuracy: 0.7375 - val_loss: 0.9140
Epoch 8/10
40/40 ━━━━━━━━━━━━━━━━━━━━ 6s 150ms/step - accuracy: 0.9969 - loss: 0.0540 - val_accuracy: 0

In [ ]:
# Preprocess the test data
X_test = tokenizer.texts_to_sequences(test_csv[text_column])
X_test = pad_sequences(X_test, maxlen=max_sequence_length, padding='post', truncating='post')

# Get the true labels from the test_labels_csv
y_test = label_encoder.transform(test_labels_csv[label_column])

# Make predictions on the test set
y_test_pred = model.predict(X_test)
y_test_labels = (y_test_pred > 0.5).astype(int)

# Save the predictions to a new dataset (as a DataFrame)
predictions_df = test_csv.copy()
predictions_df['predicted_labels'] = y_test_labels

# Save the new dataset with predictions to a CSV file
predictions_df.to_csv('/content/Test/test_predictions.csv', index=False)

# Classification report for the test set
print(classification_report(y_test, y_test_labels))


7/7 ━━━━━━━━━━━━━━━━━━━━ 0s 26ms/step
              precision    recall  f1-score   support

           0       0.76      0.82      0.79       122
           1       0.68      0.59      0.63        78

    accuracy                           0.73       200
   macro avg       0.72      0.70      0.71       200
weighted avg       0.73      0.73      0.73       200

